# Low-fidelity model of lungs connected to mechanical ventilation: simulations and dataset creation.

Using as a basis the high-fidelity lung model procedure, in the following notebook we describe the methodology to simulate a low-fidelity lung model assisted by pressure-controlled mechanical ventilation.

We begin by importing the necessary modules.


---


\
**Note**: Up to this point, it is recommended to upload to this session the `modelfunctions` and `linearregression` functions to be imported, along with `mesh.h5` file (mesh of the high-fidelity lung model).

In [ ]:
# Import and install (***it is necessary to run this cell 2 times!***)
try:
    import dolfin
    print("oui")
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

In [ ]:
!pip install meshio==4.4.6

In [ ]:
!pip install pyDOE

In [ ]:
import meshio
import dolfin
import os
import matplotlib.pyplot as plt
import numpy as np
import time
import modelfunctions
import linearregression

from ast import Interactive
from dolfin import *
from modelfunctions import solve_poroelasticity
from linearregression import regression
from sklearn import linear_model
from pyDOE import lhs

## Parameters dataset creation

1. We specify a `range` around the baseline values of the considered lung parameter values under analysis:

Lung tissue constitutive model parameters:
*   $c$
*   $\beta$
*   $c_1$
*   $c_3$

Lung permeability:
*   $k$

Spring stiffness (chest-wall effect):
*   $K_s$


\
2. We use Latin Hypercube Sampling to obtain `sample_num` number of samples of the parameter space, bounded in the specified range.


---


\
**Note**: Just for demonstrative purposes, here we only obtain 20 samples.

In [ ]:
# We consider a range of 50% around the mean
range = 50/100

# Constitutive model parameters bounds (Bir. 2019 et al.)
C_bir2019    = [-range*356.7 + 356.7 , 356.7 +range*356.7] # parameter c
Beta_bir2019 = [-range*1.075 + 1.075 , 1.075 +range*1.075] # parameter beta
C1_bir2019   = [-range*278.2 + 278.2 , 278.2 +range*278.2] # parameter c1
C3_bir2019   = [-range*5.766 + 5.766 , 5.766 +range*5.766] # parameter c3

# Mechanical parameters bounds
per = [-range*10000 + 10000 , 10000 +range*10000]          # parameter k
KKresortee = [-range*0.08 + 0.08 , 0.08 +range*0.08]       # parameter Ks

# We use Latin Hypercube Sampling approach to generate space-filling training samples.
# Number of samples
sample_num = 20
# Bounds
lb = np.array([C_bir2019[0], Beta_bir2019[0], C1_bir2019[0], C3_bir2019[0], per[0], KKresortee[0]])
ub = np.array([C_bir2019[1], Beta_bir2019[1], C1_bir2019[1], C3_bir2019[1], per[1], KKresortee[1]])
# Generation of samples
X_data = (ub-lb)*lhs(6, samples=sample_num) + lb

# We create an output vector to store the posterior simulation results (Crs and R)
Y_data = np.zeros([sample_num, 2])
print(X_data, Y_data)

[[4.72567798e+02 8.03318105e-01 1.97689886e+02 5.70984423e+00
  1.26810601e+04 5.46009340e-02]
 [3.53440877e+02 1.29376126e+00 2.25621783e+02 5.12849479e+00
  1.09438745e+04 1.18740980e-01]
 [2.98420620e+02 7.02389636e-01 2.15791894e+02 4.12123105e+00
  5.23096891e+03 1.02946187e-01]
 [4.08652636e+02 1.56815162e+00 1.76777883e+02 3.91553196e+00
  8.63183184e+03 1.12807007e-01]
 [1.89626669e+02 6.49295421e-01 3.12371391e+02 2.99866397e+00
  1.37263336e+04 1.07024797e-01]
 [4.31880003e+02 1.51189820e+00 3.98835783e+02 7.02681859e+00
  7.55876609e+03 4.06164140e-02]
 [4.60848793e+02 1.49921040e+00 3.40832262e+02 6.42723933e+00
  9.56702622e+03 6.20599628e-02]
 [5.19461244e+02 1.06311243e+00 3.28803953e+02 4.43961878e+00
  1.19233201e+04 9.74705990e-02]
 [3.75666769e+02 1.28076225e+00 4.12726999e+02 7.95657363e+00
  1.24579244e+04 8.00547506e-02]
 [4.25226692e+02 1.14072716e+00 3.76751699e+02 5.80979020e+00
  6.23581090e+03 9.29243430e-02]
 [2.81858230e+02 1.10478618e+00 1.65329840e+02 7.3

## Performing the simulations

Next, for each sample of model parameters, the problem is solved using finite elements in FEniCS. To this end we call `solve_poroelasticity`, in which the following steps are performed:

*   Load the mesh and boundaryes
*   Build function space. We use Taylor-Hood element
*   Initialize solver
*   Time-stepping loop

At the end of each simulation, we obtain arrays of pressure, airflow, and volume waveforms, along with a time array.




In [ ]:
# Measuring time
start_time = time.time()

# Simulation loop (it will execute 'sample_num' number of cases)
i = 0

while i < len(X_data):

    ii = i
    # Obtain the model parameters of the 'i' sample
    C_bir2019    = X_data[i,0]
    Beta_bir2019 = X_data[i,1]
    C1_bir2019   = X_data[i,2]
    C3_bir2019   = X_data[i,3]
    per          = X_data[i,4]
    KKresortee   = X_data[i,5]

    models=['bir2019']
    # We use these parameters in our simulation, via 'solve_poroelasticity'
    for model in models:
        tiempos,Jacob,flux,presionestodas=solve_poroelasticity('TEST',model,'low',per,KKresortee,ii,C_bir2019,Beta_bir2019,C1_bir2019,C3_bir2019)

    i += 1

print("--- %s seconds ---" % (time.time() - start_time))

## Obtaining respiratory-system compliance and airways resistance

Finally, from the ventilator signals and using the single compartment equation of motion, we obtain the respiratory-system compliance and airways resistance parameters via least-squares fitting by calling the `regression` function:

\begin{equation}
\text{P}_{\text{aw}}(t)=\frac{\text{V}(t)}{\text{C}_{\text{rs}}}+ \text{R} \dot{\text{V}}(t)+\text{PEEP}-\text{P}_{\text{mus}}(t)
\end{equation}

In [ ]:
# Simulation loop (it will execute 'sample_num' number of cases)
i = 0

while i < len(X_data):

    ii = i

    name='bir2019'

    ## We load the mechanical ventilation curves (considering both lungs)
    fluxes=fileflujos=np.load(name+str(ii)+'fluxes.npy')*8+np.load(name+str(ii)+'fluxes.npy')*8#[0:84]
    presionestodas=filepresiones=np.load(name+str(ii)+'presionestodas.npy')#+filepresionesi
    tiempos=filetiempos=np.load(name+str(ii)+'tiempos.npy')
    Jacob=filevolumenes=np.load(name+str(ii)+'volumenes.npy')*8+np.load(name+str(ii)+'volumenes.npy')*8#[0:84]
    Jacob=Jacob-Jacob[0]

    # We read the generated .npy files from both lungs
    maflujos=fluxes*60
    mapresiones=presionestodas
    matiempos=tiempos
    mavolumenes=Jacob

    # We add a 0 in the first element of the arrays (due to missing data from simulations)
    maflujos=np.concatenate((np.array([0]),np.asarray(maflujos)))
    mapresiones=np.concatenate((np.array([0]),np.asarray(mapresiones)))
    matiempos=np.concatenate((np.array([0]),np.asarray(matiempos)))
    mavolumenes=np.concatenate((np.array([0]),np.asarray(mavolumenes)))

    # We adjust the respiratory-system compliance and airways resistance from the equation of motion
    regression(maflujos,mapresiones,matiempos,mavolumenes,name,Y_data,i)

    i += 1

bir2019
Resistance(R)= (3.203457629003445, 2) cm H2O L/S, 
Compliance (Crs)= 112.43723232711235 ml/cm H2O
---------------
bir2019
Resistance(R)= (3.783915048545522, 2) cm H2O L/S, 
Compliance (Crs)= 69.73503104979156 ml/cm H2O
---------------
bir2019
Resistance(R)= (7.596405352575292, 2) cm H2O L/S, 
Compliance (Crs)= 80.34245589528902 ml/cm H2O
---------------
bir2019
Resistance(R)= (4.766424120757759, 2) cm H2O L/S, 
Compliance (Crs)= 61.74669913105019 ml/cm H2O
---------------
bir2019
Resistance(R)= (3.003229366259611, 2) cm H2O L/S, 
Compliance (Crs)= 106.58159073644714 ml/cm H2O
---------------
bir2019
Resistance(R)= (5.323488079650627, 2) cm H2O L/S, 
Compliance (Crs)= 93.31997858415096 ml/cm H2O
---------------
bir2019
Resistance(R)= (4.262304962724921, 2) cm H2O L/S, 
Compliance (Crs)= 79.34580470189096 ml/cm H2O
---------------
bir2019
Resistance(R)= (3.454309556453954, 2) cm H2O L/S, 
Compliance (Crs)= 69.66710935348667 ml/cm H2O
---------------
bir2019
Resistance(R)= (3.3079

## Display the dataset

Finally, we display the created dataset. From this, it can be observed that:

- `X_data` is an array that corresponds to the sampled lung parameters using the Latin hypercube design: each row a sample containing the values of the six parameters.

- `Y_data` is an array where a single row corresponds to the obtained respiratory-system compliance and airways resistance after simulating the corresponding sample of lung model parameters.

In [ ]:
# Print input and output arrays
print(X_data)
print(Y_data)

# Saving arrays
np.save('input_data.npy', X_data)
np.save('output_data.npy', Y_data)

[[4.72567798e+02 8.03318105e-01 1.97689886e+02 5.70984423e+00
  1.26810601e+04 5.46009340e-02]
 [3.53440877e+02 1.29376126e+00 2.25621783e+02 5.12849479e+00
  1.09438745e+04 1.18740980e-01]
 [2.98420620e+02 7.02389636e-01 2.15791894e+02 4.12123105e+00
  5.23096891e+03 1.02946187e-01]
 [4.08652636e+02 1.56815162e+00 1.76777883e+02 3.91553196e+00
  8.63183184e+03 1.12807007e-01]
 [1.89626669e+02 6.49295421e-01 3.12371391e+02 2.99866397e+00
  1.37263336e+04 1.07024797e-01]
 [4.31880003e+02 1.51189820e+00 3.98835783e+02 7.02681859e+00
  7.55876609e+03 4.06164140e-02]
 [4.60848793e+02 1.49921040e+00 3.40832262e+02 6.42723933e+00
  9.56702622e+03 6.20599628e-02]
 [5.19461244e+02 1.06311243e+00 3.28803953e+02 4.43961878e+00
  1.19233201e+04 9.74705990e-02]
 [3.75666769e+02 1.28076225e+00 4.12726999e+02 7.95657363e+00
  1.24579244e+04 8.00547506e-02]
 [4.25226692e+02 1.14072716e+00 3.76751699e+02 5.80979020e+00
  6.23581090e+03 9.29243430e-02]
 [2.81858230e+02 1.10478618e+00 1.65329840e+02 7.3